In [1]:
import os
from glob import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.distance import cdist
from scipy import stats
import pytesseract

%matplotlib inline

### Define image paths & some other items we'll need

Note the OpenCV variables in all-caps. These are just helpful variable names for values OpenCV uses internally. This style of all-caps is commonly found in C programming for enumeration data types (constants), and is used to help make programs easier to read and maintain.

See https://docs.opencv.org/4.2.0/d4/d86/group__imgproc__filter.html for more info on enumerations related to image filtering in OpenCV.

In [2]:
kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
kernel_cross = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))

In [3]:
print(kernel_cross)

[[0 1 0]
 [1 1 1]
 [0 1 0]]


**You'll need to fix the path to your images here**

In [ ]:
data_dir = '../t01/'
apt_ref_path = 'apt_ref.tif'
img_paths = glob(os.path.join(data_dir, '*.tif'))

In [ ]:
len(img_paths)

In [ ]:
apt_ref_mask = Image.open(apt_ref_path)
apt_ref_mask = np.asarray(apt_ref_mask)

In [ ]:
apt_ref_mask.shape

In [ ]:
apt_ref_c, _ = cv2.findContours(apt_ref_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
len(apt_ref_c)

In [ ]:
apt_ref_c = apt_ref_c[0]

In [ ]:
c_mask = np.zeros(apt_ref_mask.shape, dtype=np.uint8)
cv2.drawContours(c_mask, [apt_ref_c], 0, 255, 1)
plt.figure(figsize=(8, 8))
_ = plt.imshow(c_mask, cmap='gray', vmin=0, vmax=255)

### Load a test image

Load the first image in the list using the PIL library (the only usage of PIL we will need). Once loaded, I check the shape and min/max values to determine the number of channels in the image and the range of values (8-bit vs 16-bit)

In [ ]:
img_path = img_paths[0]
print(os.path.basename(img_path))

In [ ]:
img = Image.open(img_path)
img = np.asarray(img)

In [ ]:
img.shape, img.max(), img.dtype

In [ ]:
img.min(), img.max()

So we see the file is a single channel grayscale image with 16-bit pixel values. I want to work with 8-bit pixel values, so we'll scale the values down. We could have simply cast the 16-bit array to 8-bit but these operations often will automatically normalize the min/max values in the data. We don't want to alter the data other than to scale it.

First, we scale the 16-bit integers to an 8-bit range, but this creates floats. The floats are then cast to uint8.

**I'm also flipping the image horizontally after the 8-bit conversion because I found the NumPy flip method altered the original 16-bit values (don't know why)**

In [ ]:
img_8b = img / (2**8 + 1)

In [ ]:
img_8b.min(), img_8b.max()

In [ ]:
img_8b = img_8b.astype(np.uint8)

In [ ]:
img_8b = cv2.flip(img_8b, 1)

In [ ]:
img_8b.min(), img_8b.max()

In [ ]:
plt.figure(figsize=(16, 16))
_ = plt.imshow(img_8b, cmap='gray', vmin=0, vmax=255)

#### Apply slight blur

I found applying a slight blur here avoided a bunch of miniscule (1-5px) blobs in the subsequent thresholding step

In [ ]:
img_blur = cv2.GaussianBlur(img_8b, (5, 5), 1)

In [ ]:
plt.figure(figsize=(8, 8))
_ = plt.imshow(img_blur, cmap='gray', vmin=0, vmax=255)

In [ ]:
fig = plt.figure(figsize=(16, 4))
plt.xlim(0, 256)
plt.xticks(range(0, 257, 8))
_ = plt.hist(img_8b.flatten(), bins=2**8 - 2)

In [ ]:
fig = plt.figure(figsize=(16, 4))
plt.xlim(0, 256)
plt.xticks(range(0, 257, 8))
_ = plt.hist(img_blur.flatten(), bins=2**8 - 2)

#### Next, we'll compare applying the adaptive threshold before and after the blur

In [ ]:
img_edges = cv2.adaptiveThreshold(img_8b, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 7)

In [ ]:
plt.figure(figsize=(16, 16))
_ = plt.imshow(img_edges, cmap='gray', vmin=0, vmax=255)

#### The above image shows the thresholding without the blur pre-processing. Note the small "noise"-like regions found. Below is using the slight blur image that avoids these regions. I kept the non-blur image in case we wanted to pre-filter out these small regions instead of blurring them out.

In [ ]:
img_edges_blur = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 7)

In [ ]:
plt.figure(figsize=(16, 16))
_ = plt.imshow(img_edges_blur, cmap='gray', vmin=0, vmax=255)

#### Next we apply aggressive closing of the inverse mask (~img_edges). This is done to connect all the parts of the cross fiducials. As you see from above, the bright reflection in their centers causes them to be slightly broken into about 4 parts.

In [ ]:
edges_closed = cv2.morphologyEx(~img_edges_blur, cv2.MORPH_CLOSE, kernel_ellipse, iterations=1)
edges_closed = cv2.morphologyEx(edges_closed, cv2.MORPH_CLOSE, kernel_cross, iterations=3)

In [ ]:
plt.figure(figsize=(16, 16))
_ = plt.imshow(edges_closed, cmap='gray', vmin=0, vmax=255)

#### Now we find the contours as a simple list (no hierarchy)

In [ ]:
contours, hierarchy = cv2.findContours(
    edges_closed,
    cv2.RETR_LIST,
    cv2.CHAIN_APPROX_SIMPLE
)

In [ ]:
len(contours)

In [ ]:
new_img = cv2.cvtColor(img_8b, cv2.COLOR_GRAY2RGB)
cv2.drawContours(new_img, contours, -1, (0, 255, 0), 2)

plt.figure(figsize=(16, 16))
plt.imshow(new_img, cmap='gray', vmin=0, vmax=255)
plt.show()

#### Filtering out all the giant regions so we can make room to address the remaining non-fiducial regions

In [ ]:
# remove all contours larger than ~625px
max_px = 625

small_contours = []

for c in contours:
    area = cv2.contourArea(c)
    
    if area > max_px:
        continue
    
    small_contours.append(c)

In [ ]:
new_img = cv2.cvtColor(img_8b, cv2.COLOR_GRAY2RGB)
cv2.drawContours(new_img, small_contours, -1, (0, 255, 0), 2)

plt.figure(figsize=(16, 16))
plt.imshow(new_img[:, :], cmap='gray', vmin=0, vmax=255)
plt.show()

#### Next, we check the fiducials are isolated. Notice the clusters of unwanted regions that are close to each other. To remove these we dilate the whole mask and re-apply a stricter filter

In [ ]:
# fiducials are relatively isolated
# render remaining contours, dilate and re-find contours
small_c_mask = np.zeros(img_8b.shape, dtype=np.uint8)
_ = cv2.drawContours(small_c_mask, small_contours, -1, 255, -1)

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(small_c_mask, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
small_c_mask = cv2.morphologyEx(small_c_mask, cv2.MORPH_CLOSE, kernel_ellipse, iterations=5)

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(small_c_mask, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
# find contours
contours, hierarchy = cv2.findContours(
    small_c_mask,
    cv2.RETR_LIST,
    cv2.CHAIN_APPROX_SIMPLE
)

In [ ]:
len(contours)

In [ ]:
min_px = 110
max_px = 160
max_aspect_ratio = 1.2

small_contours2 = []
c_centers = []

for c in contours:
    area = cv2.contourArea(c)
    
    # check area first, this will also filter out zero area contours (avoid div by 0)
    if area > max_px or area < min_px:
        continue
    
    c_min_rect = cv2.minAreaRect(c)
    loc = c_min_rect[0]
    (c_min_w, c_min_h) = c_min_rect[1]
    c_angle = c_min_rect[2]
    
    aspect_ratio = c_min_h / c_min_w
    
    if aspect_ratio < 1:
        aspect_ratio = 1. / aspect_ratio
    
    if aspect_ratio > max_aspect_ratio:
        continue
    
    small_contours2.append(c)
    c_centers.append(loc)

In [ ]:
len(small_contours2)

### Looks good so far, however this is just on one test image. The above processing might need to be tweaked to work on many images.

In [ ]:
new_img = cv2.cvtColor(img_8b, cv2.COLOR_GRAY2RGB)
cv2.drawContours(new_img, small_contours2, -1, (0, 255, 0), 3)

plt.figure(figsize=(16, 16))
plt.imshow(new_img, cmap='gray', vmin=0, vmax=255)
plt.show()

### Next, we tackle the image rotation

This is done by assigning fiducials to a row, then finding the slope of each row. I took the mean of the slopes and then created a transformation matrix for that rotation angle and applied it to our base 8-bit image.

In [ ]:
nearest_dists = []

for loc in c_centers:
    dists = cdist([loc], c_centers)[0]
    dists.sort()
    nearest_dists.append(dists[1])

In [ ]:
nearest_dists

In [ ]:
centers_y = [cnt[1] for cnt in c_centers]

#### Here we plot a histogram of the y-axis center locations

**This or some variation could be used to QC that we have successfully isolated just the fiducials**

In [ ]:
img_h = img_8b.shape[1]

fig = plt.figure(figsize=(16, 4))
plt.xlim(0, img_h)
plt.xticks(range(0, img_h, 100))
_ = plt.hist(centers_y, bins=int(np.sqrt(img_h)))

In [ ]:
centers_y

In [ ]:
# rows are separated by roughly 220px
assigned_idx = []
centers_y = np.array(centers_y)
row_dist = 110
rows = []

for i, cy in enumerate(centers_y):
    if i in assigned_idx:
        continue
    
    row_min = cy - row_dist
    row_max = cy + row_dist
    
    in_row = np.logical_and(centers_y > row_min, centers_y < row_max)
    row_membership = np.where(in_row)
    row_members = list(row_membership[0])
    
    rows.append(row_members)
    assigned_idx.extend(row_members)

In [ ]:
rows

In [ ]:
c_centers = np.array(c_centers)

In [ ]:
# checking the indexing for finding the y-coordinate of all center locations in the first row
c_centers[rows[0]][:, 1]

In [ ]:
gradient, intercept, r_value, p_value, std_err = stats.linregress(c_centers[rows[0]])

In [ ]:
gradient, intercept

In [ ]:
np.degrees(np.arctan(gradient))

In [ ]:
r_degs = []

for r in rows:
    gradient, intercept, r_value, p_value, std_err = stats.linregress(c_centers[r])
    r_deg = np.degrees(np.arctan(gradient))
    r_degs.append(r_deg)

In [ ]:
r_degs

In [ ]:
r_deg_mean = np.mean(r_degs)

In [ ]:
r_deg_mean

In [ ]:
rows, cols = img_8b.shape

rot_mat = cv2.getRotationMatrix2D((cols/2., rows/2.), r_deg_mean, 1)
img_rot = cv2.warpAffine(img_8b, rot_mat, (cols, rows))

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(img_rot, cmap='gray', vmin=0, vmax=255)
plt.show()

#### Here I define a function to rotate a point around another point. This allows us to transform all the fiducial center locations to the rotated space.

In [ ]:
def rotate(point, origin=(0, 0), degrees=0):
    angle = np.deg2rad(-degrees)
    
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    
    return qx, qy

In [ ]:
rot_c = rotate(c_centers[29], origin=(cols/2., rows/2.), degrees=r_deg_mean)

In [ ]:
c_centers[29], rot_c

#### The rotate function works, so apply it to all fiducial centers. However, while we do this I calculate the bounding boxes for the regions of interest relative to the fiducials, e.g. the apartment row/col numbers. Additionally, I collect regions I wanted to use for applying the luminosity correction...but this didn't work well and the regions I selected are inside the apartment so wouldn't be ideal for later time points when they could be filled with cells :(  I have removed the uniformity correction code from here so you don't have to install my cv2-extras library.

In [ ]:
apt_ref.shape

In [ ]:
new_img = cv2.cvtColor(img_rot, cv2.COLOR_GRAY2RGB)
row_text_regions = []
uni_corr_regions = []
apt_regions = []

for c_center in c_centers:
    rot_c = rotate(c_center, origin=(cols/2., rows/2.), degrees=r_deg_mean)
    c_int_tup = tuple(np.round(rot_c).astype(np.int))
    
    # rect for non-uniformity samples
    rect_vert1 = (c_int_tup[0] - 80, c_int_tup[1] - 50)
    rect_vert2 = (c_int_tup[0] - 30, c_int_tup[1])
    
    uni_corr_regions.append(
        [
            rect_vert1,
            (c_int_tup[0] - 30, c_int_tup[1] - 50),
            rect_vert2,
            (c_int_tup[0] - 80, c_int_tup[1])
        ]
    )
    
    # rect for row number
    row_rect_vert1 = (c_int_tup[0] - 10, c_int_tup[1] - 128)
    row_rect_vert2 = (c_int_tup[0] + 40, c_int_tup[1] - 100)
    
    row_text_regions.append(
        img_rot[c_int_tup[1] - 128:c_int_tup[1] - 100, c_int_tup[0] - 10:c_int_tup[0] + 40]
    )
    
    # rect for col number
    col_rect_vert1 = (c_int_tup[0] - 148, c_int_tup[1] - 30)
    col_rect_vert2 = (c_int_tup[0] - 98, c_int_tup[1] - 2)
    
    # apt region
    apt_offset_x = c_int_tup[0] - apt_ref_mask.shape[1] - 10
    apt_offset_y = c_int_tup[1] - apt_ref_mask.shape[0] + 45
    apt_c = apt_ref_c + [apt_offset_x, apt_offset_y]
    
    cv2.circle(new_img, c_int_tup, 5, (0, 255, 0), -1)
    #cv2.rectangle(new_img, rect_vert1, rect_vert2, (0, 255, 0), 1)
    cv2.rectangle(new_img, row_rect_vert1, row_rect_vert2, (0, 255, 0), 1)
    cv2.rectangle(new_img, col_rect_vert1, col_rect_vert2, (0, 255, 0), 1)
    cv2.drawContours(new_img, [apt_c], 0, (0, 255, 0), 1)

plt.figure(figsize=(16, 16))
plt.imshow(new_img[300:800, 300:800], cmap='gray', vmin=0, vmax=255)
plt.show()

plt.figure(figsize=(16, 16))
plt.imshow(new_img, cmap='gray', vmin=0, vmax=255)
plt.show()

### Everything below is OCR related and is just me playing around...nothing worked satisfactorily

In [ ]:
custom_config = r'--oem 3 --psm 8 outputbase digits'

In [ ]:
for r in row_text_regions:
    plt.figure(figsize=(4, 4))
    plt.title(pytesseract.image_to_string(r, config=custom_config))
    plt.imshow(r, cmap='gray', vmin=0, vmax=255)
    plt.show()

In [ ]:
sub_img = row_text_regions[0]

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(sub_img, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
sub_img_bilat = cv2.bilateralFilter(sub_img, 3, 5, 15)

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(sub_img_bilat, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
# apply automatic Canny edge detection using the computed median
med = np.median(sub_img)
sigma = 0.33
lower = int(max(0, (1.0 - sigma) * med))
upper = int(min(255, (1.0 + sigma) * med))

sub_img_canny = cv2.Canny(sub_img, lower, upper, apertureSize=7)

In [ ]:
med

In [ ]:
fig = plt.figure(figsize=(16, 4))
plt.xlim(0, 256)
plt.xticks(range(0, 257, 8))
_ = plt.hist(sub_img.flatten(), bins=2**8 - 2)

In [ ]:
ret, sub_img_bkgd_mask = cv2.threshold(sub_img, med, 255, cv2.THRESH_BINARY)

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(sub_img_bkgd_mask, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(sub_img_canny, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
sub_img_edges = cv2.adaptiveThreshold(sub_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 0)

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(sub_img_edges, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
sub_img_edges[0:3] = 255
sub_img_edges[-3:] = 255
sub_img_edges[:, 0:2] = 255
sub_img_edges[:, -1:] = 255

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(sub_img_edges, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
sub_img_edges = np.pad(sub_img_edges, 3, constant_values=255)

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(sub_img_edges, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
pytesseract.image_to_string(sub_img_edges, config=custom_config)

In [ ]:
sub_img_closed = cv2.morphologyEx(sub_img_edges, cv2.MORPH_OPEN, kernel_cross, iterations=1)

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(sub_img_closed, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
pytesseract.image_to_string(~sub_img_closed, config=custom_config)

In [ ]:
cv2x.filter_contours_by_size?

In [ ]:
for r in row_text_regions:
    r_pre = r
    # r_pre = cv2.bilateralFilter(r, 3, 3, 15)

    sub_img_edges = cv2.adaptiveThreshold(r_pre, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 0)
    #sub_img_edges = cv2.morphologyEx(sub_img_edges, cv2.MORPH_OPEN, kernel_cross, iterations=1)
    
    #new_contours = cv2x.filter_contours_by_size(sub_img_edges, 9)
    
    #sub_img_edges = np.zeros(sub_img_edges.shape, dtype=np.uint8)
    #_ = cv2.drawContours(sub_img_edges, new_contours, -1, 255, -1)
    # sub_img_edges = ~sub_img_edges
    
#     sub_img_edges = cv2.morphologyEx(sub_img_edges, cv2.MORPH_DILATE, kernel_cross, iterations=1)
    
    sub_img_edges[0:3] = 255
    sub_img_edges[-3:] = 255
    sub_img_edges[:, 0:2] = 255
    sub_img_edges[:, -1:] = 255
    
    sub_img_edges = np.pad(sub_img_edges, 5, constant_values=255)
    
    plt.figure(figsize=(4, 4))
    plt.title(pytesseract.image_to_string(sub_img_edges, config=custom_config))
    plt.imshow(sub_img_edges, cmap='gray', vmin=0, vmax=255)
    plt.show()